<a href="https://colab.research.google.com/github/sibat119/finetune-transformer-models/blob/main/finetune_bert_with_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 7.3MB/s 
     |████████████████████████████████| 3.3MB 45.7MB/s 
     |████████████████████████████████| 901kB 47.4MB/s 


In [ ]:
dataset_url = "https://dl.fbaipublicfiles.com/glue/data/SST-2.zip"
! wget https://dl.fbaipublicfiles.com/glue/data/SST-2.zip

--2021-06-03 01:15:06--  https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7439277 (7.1M) [application/zip]
Saving to: ‘SST-2.zip.1’

SST-2.zip.1         100%[===================>]   7.09M  --.-KB/s    in 0.1s    

2021-06-03 01:15:06 (72.7 MB/s) - ‘SST-2.zip.1’ saved [7439277/7439277]



In [ ]:
import zipfile
with zipfile.ZipFile('SST-2.zip', 'r') as zip_ref:
    zip_ref.extractall('data/')

In [ ]:
def generate_examples(filepath):
    logging.info("⏳ Generating examples from = %s", filepath)
    print("⏳ Generating examples from = %s", filepath)
    with open(filepath, encoding="utf-8") as f:
        guid = 0
        tokens = []
        pos_tags = []
        chunk_tags = []
        ner_tags = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                if tokens:
                    yield guid, {
                        "id": str(guid),
                        "tokens": tokens,
                        "pos_tags": pos_tags,
                        "chunk_tags": chunk_tags,
                        "ner_tags": ner_tags,
                    }
                    guid += 1
                    tokens = []
                    pos_tags = []
                    chunk_tags = []
                    ner_tags = []
            else:
                # conll2003 tokens are space separated
                splits = line.split(" ")
                tokens.append(splits[0])
                pos_tags.append(splits[1])
                chunk_tags.append(splits[2])
                ner_tags.append(splits[3].rstrip())
        # last example
        yield guid, {
            "id": str(guid),
            "tokens": tokens,
            "pos_tags": pos_tags,
            "chunk_tags": chunk_tags,
            "ner_tags": ner_tags,
        }